# Expected Production for Solar Reporting
In this file, I will test various ways to create expected production. The results from this work will show the best way to make expected production on a system by system bases. The code will be transferred to production code that will create expected production for the entire fleet based on model inputs.

## Model Inputs
The inputs for expected production modeling are:
### 1. Annual expected production
### 2. System size
### 3. Monthly adjusted production ratio curves
### 4. Annual degradation
### 5. Weather

## Model Calculations
### 1. Calculation A
Create 365 days of data, move date prior in service date to the end of the year, copy for 25 years, group by month, apply degradation factor
### 2. Calulation B
Make empty date range, do expected production calculation for each month, special case for in service month

## Model Outputs
### 1. Metadata
### 2. Data format
The data format could be in rows = dates, columns = system ids. Or the format could be colums =[dates, systemid, value]


In [102]:
#correct
import numpy as np
import pandas as pd
from datetime import datetime
#import matplotlib as plt

In [145]:
# inputs
system_size = 1
specific_yield_per_year = 1250
in_service_date = pd.to_datetime('2020-03-15')
annual_degradation_rate = 0.005
weather_adj = [0] * 300
#weather_adj = [i*0 for i in range(301)]
mpc_file_name = 'monthly_production_curve.csv'
mpc = pd.read_csv(mpc_file_name)


In [204]:
# create dataframe of date ranges
type(in_service_date)
#first_month = pd.to_datetime({'year':in_service_date.year,'month':in_service_date.month,'day':1})
#print(pd.to_datetime(year=2020,month=3,day=1))
# check date range
first_month = pd.to_datetime(in_service_date - pd.offsets.MonthBegin(1))
date_range = pd.date_range(start=first_month, periods=301, freq='M')


In [205]:
# print date range check
date_range

DatetimeIndex(['2020-03-31', '2020-04-30', '2020-05-31', '2020-06-30',
               '2020-07-31', '2020-08-31', '2020-09-30', '2020-10-31',
               '2020-11-30', '2020-12-31',
               ...
               '2044-06-30', '2044-07-31', '2044-08-31', '2044-09-30',
               '2044-10-31', '2044-11-30', '2044-12-31', '2045-01-31',
               '2045-02-28', '2045-03-31'],
              dtype='datetime64[ns]', length=301, freq='M')

In [148]:
df = pd.DataFrame(index=date_range)

In [149]:
#datetime(2020,in_service_date.month+1,1)
datetime(2020,13%12,1)

#pd.to_timedelta('2019-01-02', unit='d')

datetime.datetime(2020, 1, 1, 0, 0)

In [150]:
# test pandas date functions
in_service_date
last_day_of_month = in_service_date + pd.tseries.offsets.DateOffset(months=1) -pd.tseries.offsets.DateOffset(days=in_service_date.day)

In [151]:
m = list(mpc['Monthly Curve'])
first_month = in_service_date.month
last_day_of_month = in_service_date + pd.tseries.offsets.DateOffset(months=1) - pd.tseries.offsets.DateOffset(days=in_service_date.day)
days_in_month = last_day_of_month.day
first_section_m1 = float((in_service_date.day - 1)/days_in_month)
last_section_m1 = float((days_in_month - in_service_date.day + 1)/days_in_month)
print('inservice date: ',in_service_date)
print('days in the month: ', days_in_month)
print('first section is day 1 to day {0} for a total of {1} days or a fraction of {2}'.format(in_service_date.day,
    in_service_date.day-1,
  float((in_service_date.day-1)/days_in_month )))
print('first half of fist month: ',first_section_m1)
print('second half of first month:', last_section_m1)
print('first half + second half: ', first_section_m1 + last_section_m1)

# make 13 months
months_13 = m[first_month:] + m[:first_month] 
months_13.append(m[first_month])
months_13[0] = float(months_13[0]) * last_section_m1
months_13[-1] = months_13[-1] * first_section_m1
print('the 13 months of montly adjusted values are: ')
print(months_13)
print('the sum of the 13 month section is: ',sum(months_13))




inservice date:  2020-03-15 00:00:00
days in the month:  31
first section is day 1 to day 15 for a total of 14 days or a fraction of 0.45161290322580644
first half of fist month:  0.45161290322580644
second half of first month: 0.5483870967741935
first half + second half:  1.0
the 13 months of montly adjusted values are: 
[0.05538709677419354, 0.1058, 0.1143, 0.1155, 0.1095, 0.0889, 0.0706, 0.0503, 0.0443, 0.0502, 0.06, 0.0878, 0.045612903225806446]
the sum of the 13 month section is:  0.9982


In [152]:
# make annual degradation
i=2
1 -(i*.005)

0.99

In [153]:
# make annual production
annual_production = system_size * specific_yield_per_year

In [166]:
# make 25 years of production
# testing one iteration of the loop
prod = [0] * 301
prod = np.array(prod)
idx= 0
i = 0
#for i in range(25):
degradation = 1 - (i*0.005)
this_year_prod = np.array([annual_production * degradation] * 13)
temp = np.array(months_13) * this_year_prod
print(this_year_prod)
print(months_13)
print(temp)
print('sum of temp = ',sum(temp))
prod[i*13:(i+1)*13] = temp
print(prod[0:25])
#prod = prod + temp


[1250. 1250. 1250. 1250. 1250. 1250. 1250. 1250. 1250. 1250. 1250. 1250.
 1250.]
[0.05538709677419354, 0.1058, 0.1143, 0.1155, 0.1095, 0.0889, 0.0706, 0.0503, 0.0443, 0.0502, 0.06, 0.0878, 0.045612903225806446]
[ 69.23387097 132.25       142.875      144.375      136.875
 111.125       88.25        62.875       55.375       62.75
  75.         109.75        57.01612903]
sum of temp =  1247.75
[ 69 132 142 144 136 111  88  62  55  62  75 109  57   0   0   0   0   0
   0   0   0   0   0   0   0]


In [233]:
# wrong answer
# loop across all years
prod = np.array([0]*325)
print('years')
for i in range(25):
    print(i,end='  ')
    degradation = 1 - (i * annual_degradation_rate)
    this_year_prod = np.array([annual_production * degradation] * 13)
    temp = np.array(months_13) * this_year_prod
    temp1 = prod[i*13]
    prod[i*13:(i+1)*13] = temp
    prod[i*13] += temp1
    #prod[i*13] = prod[i*13] + temp[0]
    #prod[i*13+1:(i+1)*13] = temp[1:]

years
0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  

In [180]:
prod

array([ 69, 132, 142, 144, 136, 111,  88,  62,  55,  62,  75, 109,  57,
        68, 131, 142, 143, 136, 110,  87,  62,  55,  62,  74, 109,  56,
        68, 130, 141, 142, 135, 110,  87,  62,  54,  62,  74, 108,  56,
        68, 130, 140, 142, 134, 109,  86,  61,  54,  61,  73, 108,  56,
        67, 129, 140, 141, 134, 108,  86,  61,  54,  61,  73, 107,  55,
        67, 128, 139, 140, 133, 108,  86,  61,  53,  61,  73, 107,  55,
        67, 128, 138, 140, 132, 107,  85,  60,  53,  60,  72, 106,  55,
        66, 127, 137, 139, 132, 107,  85,  60,  53,  60,  72, 105,  55,
        66, 126, 137, 138, 131, 106,  84,  60,  53,  60,  72, 105,  54,
        66, 126, 136, 137, 130, 106,  84,  60,  52,  59,  71, 104,  54,
        65, 125, 135, 137, 130, 105,  83,  59,  52,  59,  71, 104,  54,
        65, 124, 135, 136, 129, 105,  83,  59,  52,  59,  70, 103,  53,
        65, 124, 134, 135, 128, 104,  82,  59,  52,  58,  70, 103,  53,
        64, 123, 133, 134, 127, 103,  82,  58,  51,  58,  70, 10

In [202]:
# loop across all years
# turn on printing for debugging
prod = np.array([0]*301)
for i in range(25):
    #print(i)
    degradation = 1 - (i * annual_degradation_rate)
    this_year_prod = np.array([annual_production * degradation] * 13)
    temp = np.array(months_13) * this_year_prod
    if i == 0:
        prod[i*13:(i+1)*13] = temp
    else:
        #print(prod[i*12:(i+1)*12+1])
        #print(temp)
        #print(prod[i*12:(i+1)*12+1] + temp)
        prod[i*12:(i+1)*12+1] = prod[i*12:(i+1)*12+1] + temp
        #print(prod[i*12:(i+1)*12+1])
   

In [203]:
prod

array([ 69, 132, 142, 144, 136, 111,  88,  62,  55,  62,  75, 109, 125,
       131, 142, 143, 136, 110,  87,  62,  55,  62,  74, 109, 124, 130,
       141, 142, 135, 110,  87,  62,  54,  62,  74, 108, 124, 130, 140,
       142, 134, 109,  86,  61,  54,  61,  73, 108, 123, 129, 140, 141,
       134, 108,  86,  61,  54,  61,  73, 107, 122, 128, 139, 140, 133,
       108,  86,  61,  53,  61,  73, 107, 122, 128, 138, 140, 132, 107,
        85,  60,  53,  60,  72, 106, 121, 127, 137, 139, 132, 107,  85,
        60,  53,  60,  72, 105, 121, 126, 137, 138, 131, 106,  84,  60,
        53,  60,  72, 105, 120, 126, 136, 137, 130, 106,  84,  60,  52,
        59,  71, 104, 119, 125, 135, 137, 130, 105,  83,  59,  52,  59,
        71, 104, 119, 124, 135, 136, 129, 105,  83,  59,  52,  59,  70,
       103, 118, 124, 134, 135, 128, 104,  82,  59,  52,  58,  70, 103,
       117, 123, 133, 134, 127, 103,  82,  58,  51,  58,  70, 102, 117,
       122, 132, 134, 127, 103,  82,  58,  51,  58,  69, 102, 11

In [187]:
temp

array([ 60.92580645, 116.38      , 125.73      , 127.05      ,
       120.45      ,  97.79      ,  77.66      ,  55.33      ,
        48.73      ,  55.22      ,  66.        ,  96.58      ,
        50.17419355])

In [221]:
# combine date ranges and production into dataframe
df = pd.DataFrame(index=date_range, data=prod)
df = pd.DataFrame(index=date_range, data={'production':prod})#.rename(index='date')
df = df.rename(index={'':'date'})

In [222]:
df.head()

,production
2020-03-31,69
2020-04-30,132
2020-05-31,142
2020-06-30,144
2020-07-31,136


In [230]:
d = {'date':date_range,'prod':prod}
df = pd.DataFrame(d).set_index('date')

In [231]:
#df = df.set_index('date')
df.head()

,prod
date,
2020-03-31,69
2020-04-30,132
2020-05-31,142
2020-06-30,144
2020-07-31,136


In [ ]:
# weather issues